In [1]:
import numpy as np
import skimage.morphology
import skimage.measure
import pandas as pd
from scipy.ndimage import gaussian_filter
from scipy.spatial import ConvexHull

In [2]:
# load the annotated image
# available from the large output files data set (see README)
topactoutput = np.loadtxt('./chen2022_adult_mouse_brain_TOPACT_PVM_MAP.txt')

In [3]:
# convert to binary image
bin_image = np.zeros_like(topactoutput).astype(bool)
bin_image[np.where(topactoutput == 0)] = True

In [4]:
# some morphology
cleaned = skimage.morphology.remove_small_objects(bin_image, 1).astype(bool)
cleaned = skimage.morphology.binary_dilation(cleaned).astype('int')
labelled = skimage.morphology.label(cleaned, connectivity=2).astype('int')

In [5]:
# export labelled regions to dataframe
df = skimage.measure.regionprops_table(labelled, properties=['centroid', 'area'])
df = pd.DataFrame(df)

Now we only want cells that are in the convex hull

In [6]:
# we just need this because the centroid titles don't make nice python variable names
df['x'] = df['centroid-0']
df['y'] = df['centroid-1']

In [7]:
hullvertices = np.loadtxt('./hull_vertices.txt')
hull = ConvexHull(hullvertices)

In [8]:
def point_in_hull(point, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

In [9]:
df['inhull'] = [ point_in_hull( np.array([row.y, row.x])) for row in df.itertuples()]

In [10]:
# filter out cells not in hull + below a size threshold
df = df[(df.inhull) & (df.area > 60)]

In [11]:
df.to_csv("./pvm_loci.csv", columns=['centroid-0', 'centroid-1', 'area'], index=False)